In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from xgboost.sklearn import XGBClassifier
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import ndcg_score
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import make_scorer

In [2]:
train_users = pd.read_csv('../data/train_data.csv')

/Users/keonhoseo/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (1243) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
test_users = pd.read_csv('../data/test_data.csv')

In [4]:
train_users.fillna(-1, inplace=True)

In [5]:
y_train = train_users['country_destination']
train_users.drop(['country_destination', 'id', 'most_used_device'], axis=1, inplace=True)
x_train = train_users.values

label_encoder = LabelEncoder()
encoded_y_train = label_encoder.fit_transform(y_train)

In [6]:
x_test = test_users.drop(['id', 'most_used_device'], axis = 1).values

In [8]:
param = {
    'max_depth': 8,
    'learning_rate': 0.5,
    'verbosity': 3,
    'objective': 'multi:softprob',
    'n_jobs': -1,
    'gamma': 0,
    'min_child_weight': 1,
    'max_delta_step': 0,
    'subsample': 1,
    'colsample_bytree': 1,
    'colsample_bylevel': 1,
    'reg_alpha': 0,
    'reg_lambda': 1,
    'scale_pos_weight': 1,
    'base_score': 0.5,
    'random_state': 42
}

In [9]:
xgb_model = xgb.XGBClassifier(**param)

In [51]:
xgb_model = xgb.XGBClassifier(
    max_depth = 8,
    learning_rate = 0.5,
    verbosity = 3,
    objective = 'multi:softprob',
    n_jobs = 4,
    gamma = 0,
    min_child_weight = 1,
    max_delta_step = 0,
    subsample = 1,
    colsample_bytree = 1,
    colsample_bylevel = 1,
    reg_alpha = 0,
    reg_lambda = 1,
    scale_pos_weight = 1,
    base_score = 0.5,
    random_state = 42
)

In [ ]:
xgb_model.fit(x_train, encoded_y_train)

[21:21:48] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=8
[21:21:52] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 90 extra nodes, 0 pruned nodes, max_depth=8
[21:21:56] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 82 extra nodes, 0 pruned nodes, max_depth=8
[21:22:01] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 216 extra nodes, 0 pruned nodes, max_depth=8
[21:22:05] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 250 extra nodes, 0 pruned nodes, max_depth=8
[21:22:09] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 158 extra nodes, 0 pruned nodes, max_depth=8
[21:22:13] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 172 extra nodes, 0 pruned nodes, max_depth=8
[21:22:17] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 286 extra nodes, 0 pruned nodes, max_depth=8
[21:22:22] INFO: src/tree/updater_prune.cc:74: tree

[21:27:32] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 58 extra nodes, 0 pruned nodes, max_depth=8
[21:27:36] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 338 extra nodes, 0 pruned nodes, max_depth=8
[21:27:40] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 280 extra nodes, 0 pruned nodes, max_depth=8
[21:27:58] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 206 extra nodes, 0 pruned nodes, max_depth=8
[21:28:02] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 202 extra nodes, 0 pruned nodes, max_depth=8
[21:28:06] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 238 extra nodes, 0 pruned nodes, max_depth=8
[21:28:10] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 242 extra nodes, 0 pruned nodes, max_depth=8
[21:28:14] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 264 extra nodes, 0 pruned nodes, max_depth=8
[21:28:17] INFO: src/tree/updater_prune.cc:74: tr

[21:33:35] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 164 extra nodes, 0 pruned nodes, max_depth=8
[21:33:38] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 346 extra nodes, 0 pruned nodes, max_depth=8
[21:33:42] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 164 extra nodes, 0 pruned nodes, max_depth=8
[21:33:45] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=8
[21:33:49] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 324 extra nodes, 0 pruned nodes, max_depth=8
[21:33:52] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 134 extra nodes, 0 pruned nodes, max_depth=8
[21:34:08] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 114 extra nodes, 0 pruned nodes, max_depth=8
[21:34:12] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 164 extra nodes, 0 pruned nodes, max_depth=8
[21:34:16] INFO: src/tree/updater_prune.cc:74: t

[21:39:13] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=8
[21:39:17] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 256 extra nodes, 0 pruned nodes, max_depth=8
[21:39:21] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 128 extra nodes, 0 pruned nodes, max_depth=8
[21:39:25] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 94 extra nodes, 0 pruned nodes, max_depth=8
[21:39:29] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 180 extra nodes, 0 pruned nodes, max_depth=8
[21:39:33] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 184 extra nodes, 0 pruned nodes, max_depth=8
[21:39:36] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=8
[21:39:40] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 356 extra nodes, 0 pruned nodes, max_depth=8
[21:39:44] INFO: src/tree/updater_prune.cc:74: tre

In [53]:
xgb_model

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.5, max_delta_step=0, max_depth=8,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=4,
              nthread=None, objective='multi:softprob', random_state=42,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=3)

In [62]:
train_users.columns

Index(['age', 'signup_flow', 'age_group', 'nans', 'account_created_year',
       'account_created_month', 'account_created_week',
       'account_created_weekday', 'account_created_day', 'first_active_year',
       ...
       'sec_elapsed_sum', 'sec_elapsed_mean', 'sec_elapsed_std',
       'sec_elapsed_max', 'sec_elapsed_min', 'sec_elapsed_median',
       'day_pauses', 'long_pauses', 'short_pauses', 'session_length'],
      dtype='object', length=1265)

In [83]:
from collections import defaultdict

In [84]:
_feature_importance = defaultdict(float)
for col, importance in zip(train_users.columns, xgb_model.feature_importances_):
    _feature_importance[col] = importance

In [102]:
y_train

0           NDF
1           NDF
2            US
3         other
4            US
          ...  
213446      NDF
213447      NDF
213448      NDF
213449      NDF
213450      NDF
Name: country_destination, Length: 213451, dtype: object

In [101]:
train_users['signup_method_facebook']

0         1
1         1
2         0
3         1
4         0
         ..
213446    0
213447    0
213448    0
213449    0
213450    0
Name: signup_method_facebook, Length: 213451, dtype: int64

In [88]:
sorted(_feature_importance.items(), key = lambda x:x[1], reverse = True)

[('signup_method_facebook', 0.16539405),
 ('pending', 0.073618375),
 ('manage_listing', 0.039217062),
 ('at_checkpoint', 0.026304916),
 ('affiliate_channel_content', 0.019144962),
 ('complete', 0.012177491),
 ('first_browser_unknown', 0.011406031),
 ('age', 0.011268411),
 ('first_device_type_other_unknown', 0.011056065),
 ('gender_unknown', 0.009147341),
 ('account_created_year', 0.008865901),
 ('recommend', 0.008506723),
 ('complete_status', 0.00841488),
 ('phone_verification_number_sucessfully_submitted', 0.0071343915),
 ('signup_app_web', 0.006051299),
 ('requested', 0.006033921),
 ('signup_flow', 0.0055484925),
 ('phone_verification_number_submitted_for_call', 0.0054143565),
 ('first_active_year', 0.005327929),
 ('affiliate_provider_meetup', 0.00517468),
 ('ask_question', 0.0050822),
 ('signup_method_basic', 0.00446108),
 ('cancel', 0.0044314456),
 ('signup_app_ios', 0.0040985686),
 ('first_device_type_mac_desktop', 0.004084153),
 ('apply_coupon_error', 0.0039466564),
 ('set_user',

In [29]:
train_data = xgboost.DMatrix(x_train, encoded_y_train)

In [11]:
def ndcg5_score(preds, dtrain):
    labels = dtrain.get_label()
    top = []

    for i in range(preds.shape[0]):
        top.append(np.argsort(preds[i])[::-1][:5])

    mat = np.reshape(np.repeat(labels,np.shape(top)[1]) == np.array(top).ravel(),np.array(top).shape).astype(int)
    score = np.mean(np.sum(mat/np.log2(np.arange(2, mat.shape[1] + 2)),axis = 1))
    return 'ndcg5', score

In [30]:
param = {
    'max_depth': 8,
    'learning_rate': 1,
    'n_estimators': 5,
    'verbosity': 3,
    'objective': 'multi:softprob',
    'num_class': 12,
    'gamma': 0,
    'min_child_weight': 1,
    'max_delta_step': 0,
    'subsample': 1,
    'colsample_bytree': 1,
    'colsample_bylevel': 1,
    'reg_alpha': 0,
    'reg_lambda': 1,
    'scale_pos_weight': 1,
    'base_score': 0.5,
    'missing': None,
    'silent': True,
    'n_jobs': 4,
    'seed': 42
}

num_round = 10
xgboost.cv(param, train_data, num_boost_round=num_round, verbose_eval = True, metrics=['mlogloss'], feval=ndcg5_score)

[0]	train-mlogloss:1.19055+0.001299	train-ndcg5:0.83239+0.000176705	test-mlogloss:1.21417+0.00689788	test-ndcg5:0.827548+0.000859087
[1]	train-mlogloss:1.45821+0.00554659	train-ndcg5:0.755338+0.0024306	test-mlogloss:1.51094+0.00943521	test-ndcg5:0.744947+0.00345593
[2]	train-mlogloss:1.12708+0.0168378	train-ndcg5:0.836187+0.000897128	test-mlogloss:1.20004+0.00516024	test-ndcg5:0.822137+0.000537582
[3]	train-mlogloss:3.89355+0.0793912	train-ndcg5:0.775171+0.00761797	test-mlogloss:3.99319+0.13451	test-ndcg5:0.75786+0.00549442
[4]	train-mlogloss:1.51847+0.164038	train-ndcg5:0.836861+0.00552754	test-mlogloss:1.62851+0.15522	test-ndcg5:0.817384+0.0051638
[5]	train-mlogloss:1.94257+0.668361	train-ndcg5:0.830108+0.0141939	test-mlogloss:2.07341+0.642861	test-ndcg5:0.808339+0.0128339
[6]	train-mlogloss:1.57904+0.184578	train-ndcg5:0.841242+0.00181413	test-mlogloss:1.72138+0.159293	test-ndcg5:0.817419+0.00128098
[7]	train-mlogloss:1.63629+0.291983	train-ndcg5:0.842565+0.00361546	test-mlogloss:1.

,train-mlogloss-mean,train-mlogloss-std,train-ndcg5-mean,train-ndcg5-std,test-mlogloss-mean,test-mlogloss-std,test-ndcg5-mean,test-ndcg5-std
0,1.190554,0.001299,0.832390,0.000177,1.214168,0.006898,0.827548,0.000859
1,1.458212,0.005547,0.755338,0.002431,1.510940,0.009435,0.744947,0.003456
2,1.127079,0.016838,0.836187,0.000897,1.200042,0.005160,0.822137,0.000538
3,3.893554,0.079391,0.775171,0.007618,3.993191,0.134510,0.757860,0.005494
4,1.518466,0.164038,0.836861,0.005528,1.628515,0.155220,0.817384,0.005164
5,1.942568,0.668361,0.830108,0.014194,2.073409,0.642861,0.808339,0.012834
6,1.579043,0.184578,0.841242,0.001814,1.721384,0.159293,0.817419,0.001281
7,1.636293,0.291983,0.842565,0.003615,1.789549,0.264026,0.816488,0.002268
8,1.567674,0.292942,0.843837,0.004603,1.737137,0.272982,0.816177,0.003490
9,1.793499,0.248534,0.843362,0.001684,1.973616,0.223413,0.814036,0.000648


In [31]:
xgboost

<module 'xgboost' from '/Users/keonhoseo/opt/anaconda3/lib/python3.7/site-packages/xgboost/__init__.py'>